In [6]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_distances
from sklearn.neighbors import NearestNeighbors
import umap
import matplotlib.pyplot as plt

# =====================
# 1. Wczytaj cały zbiór danych
# =====================
df_full = pd.read_csv("../Adata/yahoo/train_subset.csv").reset_index(drop=True)
texts_full = df_full["text"].tolist()
labels_full = df_full["label"].tolist()

# Automatyczne wykrycie liczby klas
num_classes = len(set(labels_full))
print(f"📊 Liczba wykrytych klas: {num_classes}")

# =====================
# 2. Wybierz mały podzbiór do fine-tuningu
# =====================
np.random.seed(42)
samples_per_class = []

for label in df_full["label"].unique():
    class_subset = df_full[df_full["label"] == label]
    k = np.random.randint(3, 5)  # losowo 3-4 próbki na klasę
    sampled = class_subset.sample(n=min(k, len(class_subset)), random_state=42)
    samples_per_class.append(sampled)

df_finetune = pd.concat(samples_per_class).reset_index(drop=True)
texts_finetune = df_finetune["text"].tolist()
labels_finetune = df_finetune["label"].tolist()

# =====================
# 3. Przygotuj pary do kontrastowego uczenia
# =====================
examples = []
for i in range(len(texts_finetune)):
    for j in range(i + 1, len(texts_finetune)):
        label_sim = 1.0 if labels_finetune[i] == labels_finetune[j] else 0.0
        examples.append(InputExample(texts=[texts_finetune[i], texts_finetune[j]], label=label_sim))

# =====================
# 4. Fine-tuning SentenceTransformer
# =====================
base_model = SentenceTransformer('all-MiniLM-L6-v2')
train_dataloader = DataLoader(examples, shuffle=True, batch_size=16)
train_loss = losses.OnlineContrastiveLoss(model=base_model)

base_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=6,
    warmup_steps=10,
    show_progress_bar=True
)



📊 Liczba wykrytych klas: 10


100%|██████████| 198/198 [00:33<00:00,  5.89it/s]

{'train_runtime': 33.6359, 'train_samples_per_second': 94.185, 'train_steps_per_second': 5.887, 'train_loss': 0.11823393118501914, 'epoch': 6.0}


In [7]:
# # Załóżmy, że mamy listę tekstów: zbior_textow
# import pandas as pd
# import numpy as np
# from sentence_transformers import SentenceTransformer, InputExample, losses
# from torch.utils.data import DataLoader
# from sklearn.cluster import KMeans
# from sklearn.metrics import adjusted_rand_score
# from sklearn.preprocessing import normalize
# from sklearn.metrics.pairwise import cosine_distances
# from sklearn.neighbors import NearestNeighbors
# import umap
# import matplotlib.pyplot as plt

df_full = pd.read_csv("../Adata/yahoo/train_subset.csv").reset_index(drop=True)
texts_full = df_full["text"].tolist()
labels_full = df_full["label"].tolist()


from sentence_transformers import SentenceTransformer
model = base_model  # użyj wcześniej wytrenowanego modelu
embeddings = model.encode(texts_full)  # uzyskaj embeddingi dla wszystkich tekstów

# Zbuduj graf k-najbliższych sąsiadów (k-NN) na podstawie kosinusowej odległości między embeddingami
from sklearn.neighbors import NearestNeighbors
k = 5  # przykładowa liczba sąsiadów
nbrs = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(embeddings)
distances, indices = nbrs.kneighbors(embeddings)
# indices zawiera dla każdego punktu indeksy jego k+1 najbliższych (pierwszy to on sam)
knn_graph = [nbr_idx[1:] for nbr_idx in indices]  # pomijamy pierwszego (samego siebie)

# Inicjalizuj liczbę "głosów" (vote count) dla każdego kandydata na podstawie grafu
N = len(embeddings)
votes = [0] * N
for i in range(N):
    for neigh in knn_graph[i]:
        votes[neigh] += 1

M = 60  # załóżmy, że chcemy wybrać 10 przykładów
selected = []
covered = set()  # zbiór pokrytych (wybranych lub zablokowanych) indeksów

while len(selected) < M:
    # Znajdź niepokryty indeks z najwyższą liczbą głosów
    cand = None
    best_score = -1
    for idx, score in enumerate(votes):
        if idx in covered:
            continue
        if score > best_score:
            best_score = score
            cand = idx
    if cand is None:
        break  # brak kandydata (wszystko pokryte)
    # Dodaj wybrany punkt do wynikowego zbioru
    selected.append(cand)
    # Oznacz go i jego sąsiadów jako pokrytych (nie będą dalej wybierani)
    covered.add(cand)
    for neigh in knn_graph[cand]:
        covered.add(neigh)
    # Aktualizuj liczbę głosów dla pozostałych (opcjonalnie można też recomputować od zera ignorując pokryte)
    for neigh in knn_graph[cand]:
        # zmniejszamy "głosy" oddane przez lub na rzecz sąsiadów kandydata
        for nn in knn_graph[neigh]:
            if nn not in covered:
                votes[nn] -= 1

# 'selected' zawiera indeksy wybranych przykładów
print("Wybrane indeksy:", selected)


Wybrane indeksy: [303, 236, 661, 765, 811, 960, 220, 335, 815, 953, 774, 347, 24, 484, 613, 770, 951, 333, 418, 594, 874, 82, 266, 385, 439, 491, 669, 708, 763, 813, 954, 8, 532, 816, 38, 106, 134, 342, 464, 508, 675, 710, 741, 123, 308, 331, 540, 641, 692, 339, 351, 460, 487, 966, 968, 20, 64, 67, 96, 112]


In [8]:
# Utwórz nowy dataframe z wybranych przykładów
selected_df = df_full.iloc[selected].copy()

# # Zapisz do pliku CSV
# selected_df.to_csv("vote_k_selected_examples.csv", index=False)

# print("✅ Zapisano wybrane przykłady do 'vote_k_selected_examples.csv'")

print(len(pd.unique(selected_df['label'])))

selected_df.to_csv("selected_samples/yahoo/vote_k_new_model.csv", index=False)

10
